In [11]:
# Получение данных
import pandas as pd

file_name = './Hysys/data/Hysys729cases_v2.csv'  # Откуда брать файл
data_source = pd.read_csv(file_name, sep=',', index_col=0,
                          skiprows=[1]  # Пропуск единиц измерения
                          ).dropna(
    axis=1, how='all')  # Чтение файла

column_names = data_source.columns.values
# pd.options.display.float_format = '{:,.2f}'.format # Если нужно ограничить вывод до 2 знаков после запятой

# Разбиение таблицы на параметры и критерии
params = data_source.iloc[:, 0:6].astype(float)
criteria1_column = data_source.iloc[:, 6].astype(float)
criteria2_column = data_source.iloc[:, 7].astype(float)
criteria3_column = data_source.iloc[:, 8].astype(float)

print('Размер таблицы:',data_source.shape)


Размер таблицы: (729, 9)


In [12]:
# Линейная интерполяция
from scipy.interpolate import LinearNDInterpolator
import numpy as np

# Создание точек для интерполяции
points = np.array(params.values)

PRINT_PROGRESS = False  # Вывод прогресса
# Верхние и нижние границы параметров
# X_LOWER = np.array([240, 340, 90, 1.3, 60, 110])
# X_UPPER = np.array([260, 360, 110, 2, 80, 130])
X_LOWER = params.min().to_numpy()
X_UPPER = params.max().to_numpy()

print('Нижняя граница:', X_LOWER)
print('Верхняя граница:', X_UPPER)

# Создание интерполяторов
print('Расчет интерполяторов...')
print(f'Расчет интерполятора {column_names[6]}...')
criteria1 = LinearNDInterpolator(points, criteria1_column.values)
print(f'Расчет интерполятора {column_names[7]}...')
criteria2 = LinearNDInterpolator(points, criteria2_column.values)
print(f'Расчет интерполятора {column_names[8]}...')
criteria3 = LinearNDInterpolator(points, criteria3_column.values)
print('Расчет интерполяторов завершен')



Нижняя граница: [240.  340.   90.    1.3   1.    0. ]
Верхняя граница: [260. 360. 110.   2.   2. 800.]
Расчет интерполяторов...
Расчет интерполятора 10 Out - Master Comp Molar Flow (diM-Ether)...
Расчет интерполятора 15 - Master Comp Molar Flow (Methanol)...
Расчет интерполятора SPRDSHT-1 - A10: Q_sum...
Расчет интерполяторов завершен


In [6]:
# Однокритериальная оптимизация
from pymoo.algorithms.soo.nonconvex.pattern import PatternSearch  # Алгоритм Хука-Дживса
from pymoo.core.problem import Problem
from pymoo.optimize import minimize


class Best_c1 (Problem):
    def __init__(self):
        super().__init__(n_var=6, n_obj=1, n_constr=0, xl=X_LOWER, xu=X_UPPER)

    def _evaluate(self, x, out, *args, **kwargs):
        out["F"] = -criteria1(x)


class Best_c2 (Problem):
    def __init__(self):
        super().__init__(n_var=6, n_obj=1, n_constr=0, xl=X_LOWER, xu=X_UPPER)

    def _evaluate(self, x, out, *args, **kwargs):
        out["F"] = criteria2(x)


class Best_c3 (Problem):
    def __init__(self):
        super().__init__(n_var=6, n_obj=1, n_constr=0, xl=X_LOWER, xu=X_UPPER)

    def _evaluate(self, x, out, *args, **kwargs):
        out["F"] = criteria3(x)


res_c1 = minimize(Best_c1(), algorithm=PatternSearch(),
                  seed=1, verbose=PRINT_PROGRESS)
res_c2 = minimize(Best_c2(), algorithm=PatternSearch(),
                  seed=1, verbose=PRINT_PROGRESS)
res_c3 = minimize(Best_c3(), algorithm=PatternSearch(),
                  seed=1, verbose=PRINT_PROGRESS)

print('Однокритериальная оптимизация для каждого критерия (Метод Хука-Дживс)')
print(f'Первый критерий: {column_names[6]}')
print('X: ', res_c1.X)
print('F: ', res_c1.F)

print(f'Второй критерий: {column_names[7]}')
print('X: ', res_c2.X)
print('F: ', res_c2.F)

print(f'Третий критерий: {column_names[8]}')
print('X: ', res_c3.X)
print('F: ', res_c3.F)

# Сохранение результатов для однокритериальной оптимизации на основе маргинальных критериев
best_c1 = -res_c1.F
best_c2 = res_c2.F
best_c3 = res_c3.F


Однокритериальная оптимизация для каждого критерия (Метод Хука-Дживс)
Первый критерий: 10 Out - Master Comp Molar Flow (diM-Ether)
X:  [260.         360.          99.99994238   2.           2.
 799.21875   ]
F:  [-128.479063]
Второй критерий: 15 - Master Comp Molar Flow (Methanol)
X:  [260.         360.         100.00001867   2.           2.
 800.        ]
F:  [0.079176]
Третий критерий: SPRDSHT-1 - A10: Q_sum
X:  [260.  360.  110.    1.3   1.    0. ]
F:  [45872306.172231]


In [7]:
# Оптимизация на основе маргинальных решений

from pymoo.optimize import minimize
from pymoo.core.problem import Problem
from pymoo.algorithms.soo.nonconvex.pattern import PatternSearch  # Алгоритм Хука-Дживса


def objective_function(x):
    '''Маргинальная целевая функция'''
    сurrent_c1 = criteria1(x)
    сurrent_c2 = criteria2(x)
    сurrent_c3 = criteria3(x)

    diff_c1 = best_c1 - сurrent_c1
    diff_c2 = сurrent_c2 - best_c2
    diff_c3 = сurrent_c3 - best_c3

    return np.sqrt(diff_c1**2 + diff_c2**2 + diff_c3**2)


# Постановка задачи однокритериальной оптимизации
class Single_objective_optimization(Problem):
    def __init__(self) -> None:
        super().__init__(n_var=6, n_obj=1, n_constr=0, xl=X_LOWER, xu=X_UPPER)

    def _evaluate(self, x, out, *args, **kwargs):
        out["F"] = objective_function(x)


single_objective_problem = Single_objective_optimization()

# Запуск оптимизации
try:
    # Хука - Дживс
    res_PS = minimize(
        single_objective_problem, algorithm=PatternSearch(), seed=1)
    print('PS')
    print(f'X: {res_PS.X}')
    print('F: ', res_PS.F)
    print('C1: ', criteria1(res_PS.X))
    print('C2: ', criteria2(res_PS.X))
    print('C3: ', criteria3(res_PS.X))

except Exception as e:
    print('Ошибка PS')
    print(e)


PS
X: [260.  360.  110.    1.3   1.    0. ]
F:  [1.15481596]
C1:  [127.938536]
C2:  [1.099681]
C3:  [45872306.172231]


In [9]:
# Многокритериальная оптимизация
from pymoo.optimize import minimize
from pymoo.core.problem import Problem

# Алгоритмы оптимизации
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.algorithms.moo.age import AGEMOEA

# Постановка задачи


class Multi_objective_optimization(Problem):
    def __init__(self):
        super().__init__(n_var=6, n_obj=3, n_constr=0,
                         # Минимальные значения параметров
                         xl=X_LOWER,
                         # Максимальные значения параметров
                         xu=X_UPPER)

    def _evaluate(self, x, out, *args, **kwargs):
        f1 = -1 * criteria1(x)
        f2 = criteria2(x)
        f3 = criteria3(x)
        out["F"] = np.column_stack([f1, f2, f3])


# Запуск оптимизации
problem = Multi_objective_optimization()

try:
    print('Многокритериальная оптимизация (Метод NSGA2)')
    res_NSGA2 = minimize(problem, algorithm=NSGA2(), seed=1,
                         verbose=PRINT_PROGRESS, termination=('n_gen', 50))  # Вывод прогресса)
    # print('X: ', res_NSGA2.X)
    # print('F: ', res_NSGA2.F)

    # Сохранение результатов
    res_NSGA2_df = pd.DataFrame(res_NSGA2.X)
    res_NSGA2_df['dimEther_MCMF'] = -1 * res_NSGA2.F[:, 0]
    res_NSGA2_df['methanol_MCMF'] = res_NSGA2.F[:, 1]
    res_NSGA2_df['q_summ'] = res_NSGA2.F[:, 2]
    res_NSGA2_df.to_csv('NSGA2_Results.csv', sep=',', index=False,
                        header=data_source.columns.values)

except Exception as e:
    print('Ошибка NSGA2')
    print(e)

try:
    print('Многокритериальная оптимизация (Метод AGEMOEA)')
    res_AGEMOEA = minimize(problem, algorithm=AGEMOEA(),
                           seed=1, verbose=PRINT_PROGRESS,
                           # time based termination
                           termination=('time','00:03:00')
                           )

    # print('X: ', res_AGEMOEA.X)
    # print('F: ', res_AGEMOEA.F)

    # Сохранение результатов
    res_AGEMOEA_df = pd.DataFrame(res_AGEMOEA.X)
    res_AGEMOEA_df['dimEther_MCMF'] = -1 * res_AGEMOEA.F[:, 0]
    res_AGEMOEA_df['methanol_MCMF'] = res_AGEMOEA.F[:, 1]
    res_AGEMOEA_df['q_summ'] = res_AGEMOEA.F[:, 2]
    res_AGEMOEA_df.to_csv('AGEMOEA_Results.csv', sep=',', index=False,
                          header=data_source.columns.values)
except Exception as e:
    print('Ошибка AGEMOEA')
    print(e)


Многокритериальная оптимизация (Метод NSGA2)
Многокритериальная оптимизация (Метод AGEMOEA)
